# Pandoc Sidecar Demo

This notebook demonstrates how to use the pandoc/extra CLI to render LaTeX files.

## Prerequisites

Before running this demo, ensure you have the following installed:
- Docker (for containerized pandoc)

### Verify pandoc service is running and test basic functionality

`docker-compose run --rm pandoc-extra pandoc --version`

In [ ]:
import subprocess
import os

## 1. Check pandoc CLI availability

In [ ]:
cmd = ['docker-compose', 'run', '--rm', 'pandoc-extra', 'pandoc', '--version']
result = subprocess.run(cmd, capture_output=True, text=True, check=True, cwd='..')
print(result.stdout)

pandoc 3.7.0.2
Features: +server +lua
Scripting engine: Lua 5.4
User data directory: /usr/local/share/pandoc
Copyright (C) 2006-2024 John MacFarlane. Web: https://pandoc.org
This is free software; see the source for copying conditions. There is no
warranty, not even for merchantability or fitness for a particular purpose.



Pandoc outputs a standard message, which we can parse.

In [1]:
def parse_pandoc_message(message: str) -> dict:
    lines = message.strip().splitlines()
    return {
        "version": lines[0].split(" ")[1],
        "features": lines[1].split(":")[1].strip(),
        "scripting_engine": lines[2].split(":")[1].strip(),
        "user_data_directory": lines[3].split(":")[1].strip(),
        "copyright": " ".join(lines[4:])
    }

# Test the parser on an example message
print(parse_pandoc_message(
"""
pandoc 3.7.0.2
Features: +server +lua
Scripting engine: Lua 5.4
User data directory: /usr/local/share/pandoc
Copyright (C) 2006-2024 John MacFarlane. Web: https://pandoc.org
This is free software; see the source for copying conditions. There is no
warranty, not even for merchantability or fitness for a particular purpose.
"""
))

{'version': '3.7.0.2', 'features': '+server +lua', 'scripting_engine': 'Lua 5.4', 'user_data_directory': '/usr/local/share/pandoc', 'copyright': 'Copyright (C) 2006-2024 John MacFarlane. Web: https://pandoc.org This is free software; see the source for copying conditions. There is no warranty, not even for merchantability or fitness for a particular purpose.'}


## 2. Render LaTeX to PDF

note: when this is productionized, the fastapi client will need to parse the sent files, check for a main file and throw an error if missing. It should assume the other files are supporting.

In [ ]:
input_file = "input/basic_example.tex"
output_file = "output/basic_example.pdf"
workspace = os.path.abspath('.')

cmd = ['docker-compose', 'run', '--rm',
    '-v', f'{workspace}:/workspace',
    'pandoc-extra',
    f'/workspace/{input_file}',
    '-o', f'/workspace/{output_file}',
    '--pdf-engine=pdflatex']
try:
    result = subprocess.run(cmd, capture_output=True, text=True, check=True, cwd='..')
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print(f"Error rendering PDF: {e}")
    if e.stderr:
        print("Error details:")
        print(e.stderr)

## 3. Render LaTeX to HTML

In [ ]:
output_file = "output/basic_example.html"
cmd = ['docker-compose', 'run', '--rm', 
    '-v', f'{workspace}:/workspace',
    'pandoc-extra',
    f'/workspace/{input_file}',
    '-o', f'/workspace/{output_file}', 
    '--standalone', '--mathjax']

try:
    result = subprocess.run(cmd, capture_output=True, text=True, check=True, cwd='..')
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print(f"Error rendering PDF: {e}")
    if e.stderr:
        print("Error details:")
        print(e.stderr)

## 4. Render LaTeX to Markdown

In [20]:
# Render LaTeX to Markdown using pandoc CLI
output_file = "output/basic_example.md"
cmd = ['docker-compose', 'run', '--rm', 
    '-v', f'{workspace}:/workspace',
    'pandoc-extra',
    f'/workspace/{input_file}',
    '-o', f'/workspace/{output_file}', 
    '--from=latex', '--to=markdown']

try:
    result = subprocess.run(cmd, capture_output=True, text=True, check=True, cwd='..')
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print(f"Error rendering PDF: {e}")
    if e.stderr:
        print("Error details:")
        print(e.stderr)